## 조건에 맞는 댓글들 벡터화하기

- 벡터화해서 csv로 저장
- 저장된 csv 서버에서 DB로 저장

In [2]:
!pip install accelerate>=0.20.1


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
from transformers import ElectraModel, ElectraTokenizer
from transformers import ElectraForSequenceClassification
from transformers import AutoTokenizer, AutoModelForSemanticSegmentation, TrainingArguments, Trainer

import torch

import accelerate
import pandas as pd
from konlpy.tag import Mecab
import re
from tqdm import tqdm, tqdm_notebook

from sklearn.metrics import precision_recall_fscore_support, accuracy_score

In [ ]:
import transformers
transformers.__version__, accelerate.__version__

('4.30.0', '0.24.1')

### 조건에 맞는 데이터로 자르기

In [4]:
def getyear(data):
    return data.split('-')[0]

def getlen(data):
    return len(data)

# 리뷰 데이터, 테마 데이터 가져오기
rev_all = pd.read_csv('reviews.csv', index_col = 0)
theme = pd.read_csv('theme.csv', index_col = 0)

# 필요한 컬럼만 남기기
rev_all = rev_all[['id', 'themeId', 'userId','playDate', 'content']]
rev_all = rev_all.fillna('')
# 리뷰 연도 데이터 추가
rev_all['year'] = rev_all['playDate'].apply(getyear)
rev_all.head()
# 리뷰 길이 데이터 추가
rev_all['len'] = rev_all['content'].apply(getlen)

In [ ]:
display(rev_all.head())
display(theme.head())

,id,themeId,userId,playDate,content,year,len
0,836795,100,7154,2023-10-18,4인,2023,2
1,827328,100,4876,2023-09-25,#542_4명,2023,7
2,821629,100,37050,2023-09-25,230번째 방탈출\r\n2023.09.25\r\n4인 No Hint 40:30 Le...,2023,51
3,821616,100,14307,2023-09-25,#400\r\n이게 400방이라니...ㅋㅋㅋ,2023,22
4,821005,100,24039,2023-09-24,,2023,1


,id,title,company_id,intro,category_id,level,recommendPerson,tool,activity,time,grade,thumbnail,createdAt
0,67,인형괴담,334,"친구들과 술한잔 걸치고 집으로 돌아가던 어느 날, 우연히 투신 자살을 목격하게 된다...",8,3,4,2,1,60,3.7,https://roomescape-backend-image.s3.ap-northea...,2021-11-25
1,68,영혼의 편지,334,미스터M의 책상에 놓여진 의문의 편지 한통.\n\n편지를 집는 순간 코 끝에 퍼지는...,1,3,2,2,1,60,2.7,https://roomescape-backend-image.s3.ap-northea...,2021-11-25
2,69,개인병원의 비밀,334,"음침하게 소나기가 내리는 어느 날,\n\n비를 피해서 들어간 곳은 문을 닫은지 오래...",1,3,2,2,2,60,2.7,https://roomescape-backend-image.s3.ap-northea...,2021-11-25
3,70,사라진 왕관,334,"어느 날, 세계에서 가장 값비싼 왕관이 박물관에서 사라졌다.\n\n그리고 남겨진 메...",3,3,4,2,1,60,2.5,https://roomescape-backend-image.s3.ap-northea...,2021-11-25
4,81,나비효과 2,337,과거로 돌아가 아버지를 구해야 한다,7,3,2,0,0,60,2.9,https://roomescape-backend-image.s3.ap-northea...,2021-11-25


In [5]:
# 1. 최근 2년 이내 작성된 댓글 필터링(2022, 2023)
rev_all = rev_all[rev_all['year'] >= '2022']
# 2. 글자수 15 ~ 150인 댓글 필터링
rev_all = rev_all[(15 <= rev_all['len']) & (rev_all['len'] <= 150)]
# 3. 작성 댓글이 10개 이상인 유저의 댓글 필터링
u_10 = []
for id, data in rev_all.groupby('userId'):
    if len(data) >= 10:
        u_10.append(id)
rev_all = rev_all[rev_all['userId'].isin(u_10)]
# 4. 평균 평점 2.5 이상인 테마 필터링
t_id = theme[theme['grade'] >= 2.5]['id'].tolist()
rev_all = rev_all[rev_all['themeId'].isin(t_id)]
# 5. 테마에 달린 리뷰 갯수가 30개 이상인 테마 필터링
t_id = []
for id, data in rev_all.groupby('themeId'):
    if len(data) >= 30:
        t_id.append(id)
rev_all = rev_all[rev_all['themeId'].isin(t_id)]


### 모델, 토크나이저 불러오기

In [6]:
tokenizer_path = "../lm/tokenizer"
tokenizer = ElectraTokenizer.from_pretrained(tokenizer_path)


keywords = ['activity', 'creativity', 'device', 'fear', 'fun', 'guide', 'interior', 'light', 'probability', 'production', 'scale', 'service', 'story']
# 모델 딕셔너리로 만들기
models = {}
for k in keywords:
    model_path = f"../lm/models/{k}_model"
    model = ElectraForSequenceClassification.from_pretrained(model_path)
    model.resize_token_embeddings(len(tokenizer))
    models[k] = model

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


### 리뷰 데이터 토큰화

In [7]:
import pandas as pd
import re

def cleaned_content(text):
    d = re.sub('\n', '. ', text) # 줄바꿈 > .
    d = re.sub('[^가-힣0-9a-zA-Z ]{2,}', ".", d) # 특수문자 두개 이상인거 .으로 변경
    return d

r_pred_df = rev_all[['content']].astype(str)
r_pred_df['content'] = r_pred_df['content'].apply(cleaned_content)

tok_list = []
for text in r_pred_df.content:
    tok_list.append(tokenizer(text, return_tensors="pt",
            max_length=128,
            padding=True,
            truncation=True,
            add_special_tokens=True))
print('tokenize end')

tokenize end


In [ ]:
from torch.nn.functional import softmax
import torch
from tqdm import tqdm
# text : str or str[]
def classify_text(text, model):
    outputs = model(**text)

    # 소프트맥스 함수를 사용하여 확률값 계산
    probabilities = softmax(outputs.logits, dim=1)

    # 가장 높은 확률값에 해당하는 클래스 선택
    predicted_class = torch.argmax(probabilities).item() - 1

    return predicted_class, probabilities

In [ ]:

for key in keywords[10:]:
    print(f'{key} start')
    pred_li = []
    for tok in tqdm(tok_list):
        pred, _ = classify_text(tok, models[key])
        pred_li.append(pred)
    r_pred_df[key] = pred_li
    r_pred_df.to_csv('./pred.csv')
    print(f'{key} end')

light start


100%|██████████| 94357/94357 [54:29<00:00, 28.86it/s]


light end
probability start


 11%|█         | 10033/94357 [05:28<57:11, 24.57it/s]